<a href="https://colab.research.google.com/github/Krishnaugale353/Sentiment-Analysis-on-IMDB-dataset/blob/main/notebook788dc9ca28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import re
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.listdir('/kaggle/input/')

['imdb-dataset']

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train.csv')

In [ ]:
df.head()

,review,sentiment
0,That's what I kept asking myself during the ma...,negative
1,I did not watch the entire movie. I could not ...,negative
2,A touching love story reminiscent of In the M...,positive
3,This latter-day Fulci schlocker is a totally a...,negative
4,"First of all, I firmly believe that Norwegian ...",negative


In [ ]:
df.isnull().sum()

,0
review,0
sentiment,0


In [ ]:
df['review'][4]

"First of all, I firmly believe that Norwegian movies are continually getting better. From the tedious emotional films of the 70's and 80's, movies from this place actually started to contain a bit of humour. Imagine.. Actual comedies were made! Movies were actually starting to get entertaining and funny, as opposed to long, dark, depressing and boring.<br /><br />During the 90's and 00's several really great movies were made by a 'new generation' of filmmakers. Movie after movie were praised by critics and played loads of money. It became the norm!<br /><br />Then came United...<br /><br />*MINOR SPOILERS* It's just simply not funny. Not once. Not ever. But the thing is... We THINK its funny. Because we're used to norwegian movies to be funny. Especially with a cast like this with a few really funny comedians. But.. They neither say nor do anything funny! Where's the humor? Show me the humor! Is it the awkward clerk played by Harald Eia? Is it the overacting totally ridiculously unrea

**Text Preprocessing**

In [ ]:
def preprocessing_text(text):
  text = text.lower()
  text = re.sub('[^a-z ]', '', text)
  text = re.sub('<[^>]+>', '', text)
  return text

In [ ]:
df['review'] = df['review'].apply(preprocessing_text)

In [ ]:
df['review'][13]

'the cast played shakespearebr br shakespeare lostbr br i appreciate that this is trying to bring shakespeare to the masses but why ruin something so goodbr br is it because the scottish play is my favorite shakespeare i do not know what i do know is that a certain rev bowdler hence bowdlerization tried to do something similar in the victorian erabr br in other words you cannot improve perfectionbr br i have no more to write but as i have to write at least ten lines of text and english composition was never my forte i will just have to keep going and say that this movie as the saying goes just does not cut it'

Tokenizing and converting to Sequences (Integer Encoding)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

2024-07-30 10:09:17.306438: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 10:09:17.306553: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 10:09:17.420601: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
tokenizer = Tokenizer(split=" ")

In [ ]:
tokenizer.fit_on_texts(df['review'].values)

In [ ]:
x = tokenizer.texts_to_sequences(df['review'].values)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
X = pad_sequences(x)

In [ ]:
length=[]
for i in X:
  length.append(len(i))
max(length)

2460

In [ ]:
len(tokenizer.word_index)

176094

Building a Word2Vec model

In [ ]:
from gensim.models import Word2Vec

In [ ]:
reviews =[sent.split() for sent in df['review'].values]

In [ ]:
word2vec_model = Word2Vec(reviews, sg=1, window=5, vector_size=100)

In [ ]:
word2vec_model.train(reviews, total_examples=word2vec_model.corpus_count, epochs=10)

(84491352, 114571470)

Splitting the data

In [ ]:
df['sentiment'].values

array(['positive', 'positive', 'positive', ..., 'negative', 'negative',
       'negative'], dtype=object)

In [ ]:
y = np.array([1 if  i == 'positive' else 0 for i in df['sentiment']])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Creating the Embedding Matrix

In [ ]:
embedding_matrix = np.zeros((len(tokenizer.word_index)+1, 100)) #(vocab_size, vector_size)

In [ ]:
for word, index in tokenizer.word_index.items():
  if word in word2vec_model.wv:
    embedding_matrix[index] = word2vec_model.wv[word]
  else:
    embedding_matrix[index] = np.zeros(100)

Model Building

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
import keras_tuner as kt

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=2460, weights=[embedding_matrix], trainable=False))
    for i in range(hp.Int('num_layers', min_value=1, max_value=4, step=1)):
        model.add(LSTM(units=hp.Int('units'+str(i), min_value=16, max_value=128, step=8), return_sequences=True))
    model.add(LSTM(units=hp.Int('unitsf', min_value=8, max_value=64, step=8)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimizer',values=['adam','nadam']),loss='BinaryCrossentropy', metrics=['accuracy'])
    return model

In [ ]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=10, project_name='IMDB', directory='mydir7')

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
tuner.search(X_train, y_train, epochs=2, batch_size=128, validation_data=(X_test, y_test))

Trial 10 Complete [00h 04m 45s]
val_accuracy: 0.8518999814987183

Best val_accuracy So Far: 0.878000020980835
Total elapsed time: 00h 32m 32s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units0': 112,
 'unitsf': 64,
 'optimizer': 'nadam',
 'units1': 96,
 'units2': 96,
 'units3': 40}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 25 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 2460, 100)      │    17,609,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 2460, 112)      │        95,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 2460, 96)       │        80,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,260,386 (69.66 MB)

 Trainable params: 216,961 (847.50 KB)

 Non-trainable params: 17,609,500 (67.17 MB)

 Optimizer params: 433,925 (1.66 MB)

In [ ]:
model.fit(X_train, y_train, batch_size=128, epochs=10, initial_epoch=3, validation_data=(X_test, y_test))

Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 118s 368ms/step - accuracy: 0.8623 - loss: 0.3293 - val_accuracy: 0.8677 - val_loss: 0.3060
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 114s 365ms/step - accuracy: 0.8765 - loss: 0.2977 - val_accuracy: 0.8886 - val_loss: 0.2688
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 114s 364ms/step - accuracy: 0.8868 - loss: 0.2802 - val_accuracy: 0.8605 - val_loss: 0.3280
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 116s 369ms/step - accuracy: 0.8112 - loss: 0.4058 - val_accuracy: 0.8938 - val_loss: 0.2530
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 116s 372ms/step - accuracy: 0.8975 - loss: 0.2527 - val_accuracy: 0.9062 - val_loss: 0.2315
Epoch 9/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 116s 371ms/step - accuracy: 0.9073 - loss: 0.2314 - val_accuracy: 0.8972 - val_loss: 0.2424
Epoch 10/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 116s 371ms/step - accuracy: 0.9137 - loss: 0.2192 - val_accuracy: 0.9118 - val_loss: 0.2195


In [ ]:
model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 25s 80ms/step - accuracy: 0.9108 - loss: 0.2203


[0.21945638954639435, 0.9118000268936157]

In [ ]:
model.save('sentiment.h5')

In [ ]:
import pickle

In [ ]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)